In [22]:
import os
from dotenv import load_dotenv
from IPython.display import display,Markdown
from openai import OpenAI
import json
from scraper import fetch_website_contents,fetch_website_links

In [23]:
## initialize env file
load_dotenv(override=True)
api_key=os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print('API key looks good so far')
else:
    print('There might be a problem with your API key? Please visit the troubleshooting notebook!')


API key looks good so far


In [24]:
MODEL="gpt-5-nano"
openai=OpenAI()

In [25]:
links=fetch_website_links('https://edwarddonner.com')
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-

In [26]:
#First Step "System Prompt"

link_system_prompt=""" 
you are provided with a list of links found on a webpage.
you are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": 'about page',"url": "https://full.url/goes/here/about"},
        {"type": "careers page","url": "https://another.full.url/careers"}
    ]
}

"""



In [27]:
def get_links_user_prompt(url):
    user_prompt=f""" 
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
    """
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [28]:
print(get_links_user_prompt("https://edwarddonner.com"))

 
Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company,
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
    https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2025/11/

In [29]:
def select_relevant_links(url):
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':link_system_prompt},
            {'role':'user','content':get_links_user_prompt(url)}
        ],
        response_format={'type':'json_object'}
    )
    result=response.choices[0].message.content
 #   print(result)
    links=json.loads(result)
    #print(links)
    return links

In [30]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links

In [31]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'external partner page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [35]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [36]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 9 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [37]:
### Second step: Make the brochure

In [38]:
def fetch_page_and_all_relevant_links(url):
    contents=fetch_website_contents(url)
    relevant_links=select_relevant_links(url)
    result=f"## Landing page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [39]:
print(fetch_page_and_all_relevant_links("https://edwarddonner.com"))

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 9 relevant links
## Landing page:

Home - Edward Donner

Home
AI Curriculum
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
If left unchecked, I will happily drone on about LLMs to anyone that will listen. My friends got fed up with my impromptu lectures, and convinced me to make some Udemy courses. To 

In [40]:
### 3rd step to create brochur

In [41]:
brochure_system_prompt=""" 
you are an assistant that analyzes the contents of several relevant pages from a company website 
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [42]:
def get_brochure_user_prompt(company_name,url):
    user_prompt=f""" 
You are looking at a company called {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
    """
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000]
    return user_prompt

In [43]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


" \nYou are looking at a company called HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n    ## Landing page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nLightricks/LTX-2\nUpdated\n2 days ago\n•\n330k\n•\n703\ntencent/HY-MT1.5-1.8B\nUpdated\n9 days ago\n•\n8.05k\n•\n696\nQwen/Qwen-Image-2512\nUpdated\n10 days ago\n•\n19.7k\n•\n552\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n2 days ago\n•\n3.15k\n•\n261\nnvidia/nemotron-speech-streaming-en-0.6b\nUpdated\n4 days ago\n•\n1.07k\n•\n256\nBrowse 2M+ models\nSpaces\nRunning\nFeatur

In [46]:
def create_brochure(company_name,url):
    response=openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=[{"role":"system","content":brochure_system_prompt},
        {"role":"user","content":get_brochure_user_prompt(company_name,url)}
                    
             ]
    )
    result=response.choices[0].message.content
    display(Markdown(result))


In [48]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the vibrant AI community shaping the future of machine learning. As a leading collaboration platform, Hugging Face empowers machine learning engineers, scientists, and users worldwide to create, discover, and share cutting-edge models, datasets, and AI applications. With a commitment to openness and ethics, Hugging Face fosters a global ecosystem where innovation thrives.

---

## What We Offer

- **2 Million+ Models:** Explore a vast collection of state-of-the-art public machine learning models including text, image, video, audio, and even 3D modalities. Our trending models, updated regularly, showcase cutting-edge advancements in AI.
  
- **500K+ Datasets:** Access and contribute to a rich repository of diverse datasets that fuel training and research across industries.
  
- **Spaces:** Host and interact with AI-powered applications directly within the platform, from image generation to complex data visualizations.
  
- **Open Source Stack:** Leverage our open-source tools and libraries to accelerate your ML projects, improve workflows, and collaborate seamlessly.

- **Enterprise Solutions:** Benefit from paid compute resources and tailored enterprise offerings for scalable AI development.

---

## Our Community & Culture

Hugging Face is more than a platform—it's a thriving **community** of ML enthusiasts and professionals dedicated to transparent collaboration and continuous learning. We emphasize:

- **Openness & Sharing:** Everyone can host models, datasets, and applications publicly, fostering collective growth.
- **Ethical AI:** Building responsible AI technologies with fairness and inclusivity at the core.
- **Learning & Portfolio Building:** Users can showcase their ML projects and build strong profiles recognized by peers and industry leaders.
- **Innovation:** Constantly pushing the boundaries in AI research and development.

---

## Who Uses Hugging Face?

Our platform supports a diverse user base including:

- Machine Learning Researchers
- Data Scientists
- AI Developers
- Enterprises seeking customized AI solutions
- Educators and Students
- Open Source Contributors  
Organizations across tech, research, entertainment, and many other sectors leverage Hugging Face to power their ML initiatives.

---

## Careers at Hugging Face

Join us to work at the forefront of AI innovation. Hugging Face offers career opportunities in areas such as:

- Machine Learning Engineering
- Research Science
- Software Development
- Community Management
- Product and Design

We seek passionate individuals who thrive in open, collaborative environments and want to impact the future of AI. Work with a global team committed to building ethical, accessible, and powerful AI technologies.

---

## Get Started

- **Explore Models & Applications:** Browse millions of models and AI apps or try out trending AI demos on the platform.
- **Host Your Work:** Share your models, datasets, and app spaces with the world.
- **Sign Up:** Create your profile to join the global ML community and boost your projects.
- **Enterprise Solutions:** Contact us for tailored compute and AI services.

---

## Brand & Identity

- Logo colors: Bright Yellow (#FFD21E), Warm Orange (#FF9D00), and Neutral Gray (#6B7280)
- The Hugging Face Hub is the go-to destination for open-source ML collaboration.

---

**Hugging Face**  
*The AI community building the future.*

Website: https://huggingface.co  
Join us and be part of the #MachineLearningRevolution!

---

*This brochure compiles the latest information about Hugging Face’s mission, offerings, community, and career opportunities to guide prospective customers, investors, and recruits.*